<a href="https://colab.research.google.com/github/manojmanivannan/ApacheSparkEssentials/blob/master/SparkStreamByPluralsight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!apt-get update
!apt-get install -y openjdk-8-jdk-headless scala jq  ssh > /dev/null
!if ! [[ -f "spark-2.3.1-bin-hadoop2.7.tgz" ]]; then wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz; tar xf spark-2.3.1-bin-hadoop2.7.tgz; fi
!if ! [[ -f "ngrok-stable-linux-amd64.zip" ]]; then wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip; unzip ngrok-stable-linux-amd64.zip; fi
!if ! [[ -f "goodls_linux_amd64" ]]; then wget -q https://github.com/tanaikech/goodls/releases/download/v2.0.1/goodls_linux_amd64; chmod +x goodls_linux_amd64; fi
!cat /etc/ssh/ssh_host_rsa_key.pub > /etc/ssh/authorized_keys
!apt-get install python3.7 python3-pip

!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!python3 -m pip install pyspark findspark
# !update-alternatives --config python3

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (99.7 kB/s)
Reading package li

In [22]:
# !export SPARK_HOME="/content/spark-2.3.1-bin-hadoop2.7"
# !export PATH="$PATH:$SPARK_HOME/bin"

In [23]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


In [34]:
!/content/spark-2.3.1-bin-hadoop2.7/sbin/stop-master.sh
!/content/spark-2.3.1-bin-hadoop2.7/sbin/stop-slave.sh
!/content/spark-2.3.1-bin-hadoop2.7/sbin/start-master.sh

stopping org.apache.spark.deploy.master.Master
no org.apache.spark.deploy.worker.Worker to stop
starting org.apache.spark.deploy.master.Master, logging to /usr/local/lib/python3.8/dist-packages/pyspark/logs/spark--org.apache.spark.deploy.master.Master-1-8afaf94dcefa.out


In [35]:
!sleep 3 && grep -oP "MasterUI' on port ([0-9]+)" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master*.out 
!grep -oP "spark:.*" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master*.out 

MasterUI' on port 8081
spark://8afaf94dcefa:7077


In [36]:
!/content/spark-2.3.1-bin-hadoop2.7/sbin/start-slave.sh $(grep -oP "spark:.*" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master*.out)
!sleep 3 && grep -oP "WorkerUI' on port ([0-9]+)" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.worker*.out 

starting org.apache.spark.deploy.worker.Worker, logging to /usr/local/lib/python3.8/dist-packages/pyspark/logs/spark--org.apache.spark.deploy.worker.Worker-1-8afaf94dcefa.out
WorkerUI' on port 8082


In [37]:
get_ipython().system_raw('./ngrok authtoken 2IeJHLKYs6eB68PKJQM1VkpPu00_5JXQQZW3hofy6Q6vqTX2')
from time import sleep
def ngrok_tunnel_by_port(port: int):
  get_ipython().system_raw('pkill -f ngrok')
  get_ipython().system_raw(f'./ngrok http {port} &')
  sleep(4)
  print(get_ipython().getoutput('curl -s http://localhost:4040/api/tunnels',split=True))



In [38]:
!jps

26087 Jps
26026 Worker
21067 SparkSubmit
25951 Master


In [39]:
master_url = get_ipython().getoutput('grep -oP "spark:.*" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master*.out',split=True)
print('Master URL:',master_url[0])
# get_ipython().system_raw(f'nohup pyspark --master {master_url[0]} --num-executors 5 --driver-memory 4g --executor-memory 4g &' )

Master URL: spark://8afaf94dcefa:7077


In [40]:
!jps

26026 Worker
21067 SparkSubmit
26108 Jps
25951 Master


In [41]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

# Spark session & context
conf = SparkConf().set("spark.master",master_url[0]).set('spark.ui.port', '4050').set("spark.executor.instances", 4).set("spark.executor.cores", 4).set("spark.driver.memory",'4g').set("spark.executor.memory",'4g')
try:
  sc = SparkContext(conf=conf)
except ValueError:
  sc.stop()
  sc = SparkContext(conf=conf)

spark = SparkSession.builder.appName('PluralSight').getOrCreate()

In [42]:
ngrok_tunnel_by_port(4050)

['{"tunnels":[{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://76b8-34-139-46-169.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://76b8-34-139-46-169.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}']


In [43]:
spark

In [44]:
happiness_df = spark.read.format('csv').option('header','true').load('2016.csv')

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.8/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
!jps